# In this tutorial, you will learn the basics of Spark dataframes.

### Resources:
### Spark website: https://spark.apache.org/
### Spark SQL and DataFrames documentation: https://spark.apache.org/docs/latest/sql-programming-guide.html

## About dataframes:
### - a data structure that holds data in columns and rows (like in a relational database.)  
### - able to input and output data from a wide variety of sources
### - can apply transformations on the data


## 1. create a spark session and import dataset

In [54]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Basics').getOrCreate()

In [55]:
df = spark.read.csv('sales_info.csv',inferSchema=True,header=True)

In [56]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [57]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [58]:
df.columns

['Company', 'Person', 'Sales']

In [59]:
df.describe().show()

+-------+-------+-------+------------------+
|summary|Company| Person|             Sales|
+-------+-------+-------+------------------+
|  count|     12|     12|                12|
|   mean|   null|   null| 360.5833333333333|
| stddev|   null|   null|250.08742410799007|
|    min|   APPL|  Chris|             120.0|
|    max|   MSFT|Vanessa|             870.0|
+-------+-------+-------+------------------+



## 2. Read and display data

In [60]:
type(df['Sales'])

pyspark.sql.column.Column

In [61]:
df.select('Sales').show()

+-----+
|Sales|
+-----+
|200.0|
|120.0|
|340.0|
|600.0|
|124.0|
|243.0|
|870.0|
|350.0|
|250.0|
|130.0|
|750.0|
|350.0|
+-----+



In [62]:
df.head(5)

[Row(Company='GOOG', Person='Sam', Sales=200.0),
 Row(Company='GOOG', Person='Charlie', Sales=120.0),
 Row(Company='GOOG', Person='Frank', Sales=340.0),
 Row(Company='MSFT', Person='Tina', Sales=600.0),
 Row(Company='MSFT', Person='Amy', Sales=124.0)]

In [63]:
df.head(5)[0]

Row(Company='GOOG', Person='Sam', Sales=200.0)

In [64]:
df.select(['Company','Sales']).show()

+-------+-----+
|Company|Sales|
+-------+-----+
|   GOOG|200.0|
|   GOOG|120.0|
|   GOOG|340.0|
|   MSFT|600.0|
|   MSFT|124.0|
|   MSFT|243.0|
|     FB|870.0|
|     FB|350.0|
|   APPL|250.0|
|   APPL|130.0|
|   APPL|750.0|
|   APPL|350.0|
+-------+-----+



In [65]:
df.withColumn('double_sales',df['Sales']*2).show()

+-------+-------+-----+------------+
|Company| Person|Sales|double_sales|
+-------+-------+-----+------------+
|   GOOG|    Sam|200.0|       400.0|
|   GOOG|Charlie|120.0|       240.0|
|   GOOG|  Frank|340.0|       680.0|
|   MSFT|   Tina|600.0|      1200.0|
|   MSFT|    Amy|124.0|       248.0|
|   MSFT|Vanessa|243.0|       486.0|
|     FB|   Carl|870.0|      1740.0|
|     FB|  Sarah|350.0|       700.0|
|   APPL|   John|250.0|       500.0|
|   APPL|  Linda|130.0|       260.0|
|   APPL|   Mike|750.0|      1500.0|
|   APPL|  Chris|350.0|       700.0|
+-------+-------+-----+------------+



In [66]:
df.withColumnRenamed('Sales','new_sales').show()

+-------+-------+---------+
|Company| Person|new_sales|
+-------+-------+---------+
|   GOOG|    Sam|    200.0|
|   GOOG|Charlie|    120.0|
|   GOOG|  Frank|    340.0|
|   MSFT|   Tina|    600.0|
|   MSFT|    Amy|    124.0|
|   MSFT|Vanessa|    243.0|
|     FB|   Carl|    870.0|
|     FB|  Sarah|    350.0|
|   APPL|   John|    250.0|
|   APPL|  Linda|    130.0|
|   APPL|   Mike|    750.0|
|   APPL|  Chris|    350.0|
+-------+-------+---------+



## 3. Interact with SQL

In [68]:
df.createOrReplaceTempView('top sales')

In [69]:
results = spark.sql('select * from sales where sales>=600')

In [70]:
results.show()

+-------+------+-----+
|Company|Person|Sales|
+-------+------+-----+
|   MSFT|  Tina|600.0|
|     FB|  Carl|870.0|
|   APPL|  Mike|750.0|
+-------+------+-----+



## 4. Use group by and aggregate

In [26]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [27]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [28]:
group_data = df.groupBy('Company')

In [29]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



## 5. Import functions

In [30]:
from pyspark.sql.functions import countDistinct,avg,stddev,format_number

In [32]:
df.select(countDistinct('Person').alias('Distinct Persons')).show()

+----------------+
|Distinct Persons|
+----------------+
|              12|
+----------------+



In [33]:
sales_std = df.select(stddev('Sales').alias('std'))

In [39]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [34]:
sales_std.select(format_number('std',2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [40]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [41]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## 6. Working with Dates and Times

In [71]:
from pyspark.sql import SparkSession

In [72]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [73]:
df = spark.read.csv('appl_stock.csv',header=True,inferSchema=True)

In [74]:
df.select(['Date','Open']).show()

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [75]:
from pyspark.sql.functions import (dayofmonth,hour,dayofyear,month,
                                  year,weekofyear,format_number,date_format)

In [76]:
df.select(dayofmonth(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [77]:
df.select(month(df['Date'])).show()

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          2|
+-----------+
only showing top 20 rows



In [78]:
newdf = df.withColumn('Year',year(df['Date']))

In [79]:
result = newdf.groupBy('Year').mean().select(['Year','avg(Close)'])

In [80]:
result.show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [81]:
new_result = result.withColumnRenamed('avg(Close)','Average closing price')

In [83]:
new_result.select(['Year',format_number('Average closing price',2).
                   alias('Avg Close')]).orderBy('Year').show()

+----+---------+
|Year|Avg Close|
+----+---------+
|2010|   259.84|
|2011|   364.00|
|2012|   576.05|
|2013|   472.63|
|2014|   295.40|
|2015|   120.04|
|2016|   104.60|
+----+---------+

